In [4]:
import yaml
import importlib

config = yaml.load(open("gaia_ae_8_trunc_normal.yaml"), Loader=yaml.FullLoader)
model_class_path = config["model"]["class_path"]
module_name, class_name = model_class_path.rsplit(".", 1)
module = importlib.import_module(module_name)
model_class = getattr(module, class_name)
model_init_args = config["model"]["init_args"]
model = model_class(**model_init_args)
model

AutoencoderPure()

In [3]:
# Print parameter summary
print("Model Parameter Summary:")
print("-" * 80)
print(f"{'Module':<40} {'Shape':<20} {'Requires Grad':<15} {'Params':<10}")
print("-" * 80)

total_params = 0
trainable_params = 0

for name, param in model.named_parameters():
    param_count = param.numel()
    total_params += param_count
    if param.requires_grad:
        trainable_params += param_count

    print(
        f"{name:<40} {str(list(param.shape)):<20} {str(param.requires_grad):<15} {param_count:<10,}"
    )

print("-" * 80)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {total_params - trainable_params:,}")

Model Parameter Summary:
--------------------------------------------------------------------------------
Module                                   Shape                Requires Grad   Params    
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Total parameters: 0
Trainable parameters: 0
Non-trainable parameters: 0
